In [100]:
import xarray as xr
import torch
from torch.utils.data import DataLoader
from xbatcher import BatchGenerator

import matplotlib.pyplot as plt
from utils.general import load_config

config = load_config()

In [119]:
hr_data = xr.open_dataset(
    "/home/ubuntu/project/destine-super-resolution/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr",
    engine="zarr",
    chunks={})

lr_data = xr.open_dataset(
    "/home/ubuntu/project/destine-super-resolution/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-standard-sfc-v0.zarr",
    engine="zarr",
    chunks={})

In [120]:
hr_data

<xarray.Dataset> Size: 64GB
Dimensions:           (time: 240, latitude: 4096, longitude: 8193)
Coordinates:
    entireAtmosphere  float64 8B ...
  * latitude          (latitude) float64 33kB -90.0 -89.96 -89.91 ... 89.96 90.0
  * longitude         (longitude) float64 66kB -180.0 -180.0 ... 180.0 180.0
    step              timedelta64[ns] 8B ...
    surface           float64 8B ...
  * time              (time) datetime64[ns] 2kB 2020-01-01 ... 2020-01-10T23:...
Data variables:
    d2m               (time, latitude, longitude) float32 32GB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    t2m               (time, latitude, longitude) float32 32GB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-11-14T08:10 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [121]:
hr_data = hr_data[config['dataset']['data_target']]
lr_data = lr_data[config['dataset']['data_variable']]

In [123]:
GPU_DEVICE = 0
device = torch.device("cuda",GPU_DEVICE)
batch_generator_lr = BatchGenerator(lr_data, input_dims={"time": config['training']['batch_size'], "latitude":  512, "longitude": 1025})
batch_generator_hr = BatchGenerator(hr_data, input_dims={"time": config['training']['batch_size'], "latitude":  512, "longitude": 1025})


In [ ]:

# Iterate through one batch
for batch in batch_generator_lr:

    data  = batch.load()
    print(data.sizes)
    data = data.to_array().values
    data = torch.tensor(data)

    data = torch.permute(data, (1, 0, 2, 3))
    print(data.shape)
    data.to(device)
    break

Frozen({'time': 16, 'latitude': 512, 'longitude': 1025})
torch.Size([16, 3, 512, 1025])


In [ ]:
# cloud based 20.3 s ± 437 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) (24 x 4096 x 8193)
# 3.23 s ± 41.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) (24 x 4096 x 8193)

In [124]:
# Iterate through both batch generators together
for batch_lr, batch_hr in zip(batch_generator_lr, batch_generator_hr):

    # Load LR and HR batches into memory
    lr_data = batch_lr.load().to_array().values
    hr_data = batch_hr.load().to_array().values

    # Convert to PyTorch tensors
    lr_tensor = torch.tensor(lr_data, dtype=torch.float32).to(device)
    hr_tensor = torch.tensor(hr_data, dtype=torch.float32).to(device)

    hr_tensor = torch.permute(hr_tensor, (1, 0, 2, 3))
    lr_tensor = torch.permute(lr_tensor, (1, 0, 2, 3))

    print("LR Batch Shape:", lr_tensor.shape)  # Expected: (num_vars, batch_size, lat, lon)
    print("HR Batch Shape:", hr_tensor.shape)  # Expected: (num_vars, batch_size, lat, lon)

    break  # Remove this if you want to iterate through all batches

LR Batch Shape: torch.Size([16, 3, 512, 1025])
HR Batch Shape: torch.Size([16, 1, 512, 1025])


In [91]:
from loguru import logger

import torch
import torch.nn.functional as F
from torchmetrics.image.ssim import StructuralSimilarityIndexMeasure
import numpy as np
from tqdm import tqdm

In [92]:
def count_model_parameters(model):
    """
    Count number of parameters in a Pytorch Model.

    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [94]:
def z_score_normalization(tensor: torch.Tensor) -> torch.Tensor:
    """
    Perform Z-score normalization for each channel of the input tensor.

    Args:
        tensor (torch.Tensor): Input tensor of shape [C, H, W].

    Returns:
        torch.Tensor: Normalized tensor.
    """
    mean = tensor.mean(dim=[1, 2], keepdim=True)  # Mean along H and W for each channel
    std = tensor.std(dim=[1, 2], keepdim=True)    # Standard deviation along H and W for each channel

    # Prevent division by zero
    std = std + 1e-6
    return (tensor - mean) / std

In [ ]:

upscale = 8
window_size = 8

model = SwinIR(in_chans=3, upscale=upscale,
                window_size=window_size, img_range=1., depths=[6, 6, 6, 6],
                embed_dim=60, num_heads=[6, 6, 6, 6], mlp_ratio=2, upsampler='pixelshuffledirect')


ModuleNotFoundError: No module named 'models.model'

In [62]:
## opt technic

In [55]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import xarray as xr

import xbatcher as xb
import xbatcher.loaders.torch
import xbatcher

In [68]:
hr_data

<xarray.Dataset> Size: 32GB
Dimensions:           (time: 240, latitude: 4096, longitude: 8193)
Coordinates:
    entireAtmosphere  float64 8B ...
  * latitude          (latitude) float64 33kB -90.0 -89.96 -89.91 ... 89.96 90.0
  * longitude         (longitude) float64 66kB -180.0 -180.0 ... 180.0 180.0
    step              timedelta64[ns] 8B ...
    surface           float64 8B ...
  * time              (time) datetime64[ns] 2kB 2020-01-01 ... 2020-01-10T23:...
Data variables:
    t2m               (time, latitude, longitude) float32 32GB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-11-14T08:10 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [84]:
X_bgen = xb.BatchGenerator(
    lr_data[['t2m','u10','v10']],
    input_dims={"time": config['training']['batch_size'],"latitude":  512, "longitude": 1025},
    preload_batch=True,
)

y_bgen = xb.BatchGenerator(
    hr_data['t2m'],
    input_dims={"time": config['training']['batch_size'],"latitude":  512, "longitude": 1025},
    preload_batch=True,
)

In [88]:
dataset = xbatcher.loaders.torch.IterableDataset(X_bgen, y_bgen)